In [ ]:
import torch
torch.manual_seed(7)
from dataloader import StackExchangeXMLDataset


In [ ]:
dataset = StackExchangeXMLDataset('data/datasciencestackexchangepostsmin.xml')

In [ ]:
# print(dataset.train_data)
X = dataset.prepare_data_for_model(dataset.train_data)

In [ ]:
xb,yb = dataset.get_batch(X,3)
#c_idxs = [r.index(data.TOKEN_MAP["<|endofprompt|>"]) for r in sample]

In [ ]:
xb.size(),yb.size()

In [ ]:
import torch.nn as nn 
from torch.nn import functional as F 


class BiGramLM(nn.Module):
    def __init__(self,vocab_size):
        super().__init__() 
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size) 
    
    
    def forward(self,idx,targets):
        
        logits = self.token_embedding_table(idx)
        B,T,C = logits.shape
        logits = logits.view(B*T,C)
        targets = targets.view(-1) # B*T
        loss = F.cross_entropy(logits,targets)
        

        return logits,loss

m = BiGramLM(dataset.enc.n_vocab)
logits,loss = m(xb,yb)
print(logits.shape)
print(loss)

        

In [ ]:
print(dataset.enc.decode(list(yb[1])))
print("____")
dataset.enc.decode(list(xb[1]))
# for i in range(mini_x.size()[0]):
#     for j in range(mini_y.size()[-1]):
#         print(j)
#         context = mini_x[i,:j+ mini_x.size()[-1] - mini_y.size()[-1]]
#         target = mini_y[i,j - mini_y.size()[-1]]
#         print(f"when the context is: {dataset.enc.decode(list(context))}, the target is: {dataset.enc.decode([target.item()])} ")

In [ ]:
dataset.enc.encode("<|fim_middle|>",allowed_special={"<|fim_middle|>"})